# 🎵 Music Converter Backend on Colab

此笔记本用于在 Google Colab 的免费 GPU 上运行 Music Converter 后端。

**前置准备：**
1. 拥有一个 [Ngrok](https://dashboard.ngrok.com/get-started/your-authtoken) 账号并获取 Authtoken。

In [ ]:
#  1. 初始化环境与获取代码
import os

# 检查 GPU
!nvidia-smi

# === 配置区域 ===
REPO_URL = "https://github.com/dieWehmut/music-converter.git"
BRANCH = "main"
# ================

# 克隆代码
if not os.path.exists("music-converter"):
    !git clone -b $BRANCH $REPO_URL music-converter
    %cd music-converter
else:
    %cd music-converter
    !git pull

print("✅ 代码准备就绪")

In [ ]:
# 2. 安装系统与 Python 依赖 (耗时约 2-3 分钟)

# 1. 安装系统级音频库
!apt-get update -qq
!apt-get install -y libsndfile1 ffmpeg

# 2. 安装 Python 库
# 使用清华源可能在 Colab 较慢，Colab 默认源通常很快
!pip install -r backend/requirements.txt
!pip install pyngrok uvicorn

print("✅ 依赖安装完毕")

In [ ]:
# 3. 启动后端服务
import os
import threading
import time
import subprocess
from pyngrok import ngrok

# ==========================================
# 🔐 请在这里填入你的 Ngrok Token
# 获取地址: https://dashboard.ngrok.com/get-started/your-authtoken
NGROK_AUTHTOKEN = "这里粘贴你的Token"
# ==========================================

# 1. 设置环境变量
os.environ["MC_ENABLE_LONG_AUDIO"] = "1"  # 允许长音频 (Colab 有 GPU，不用怕)
os.environ["MC_DEV_MODE"] = "0"           # 关闭 Mock 模式，使用真实模型

# 2. 配置 Ngrok
if not NGROK_AUTHTOKEN.startswith("1") and not NGROK_AUTHTOKEN.startswith("2"):
    print("❌ 错误: 请先填入正确的 Ngrok Authtoken")
else:
    ngrok.set_auth_token(NGROK_AUTHTOKEN)

    # 3. 开启隧道
    # 终止旧的隧道（如果有）
    ngrok.kill()
    
    # 建立 HTTP 隧道指向 8000 端口
    try:
        public_url = ngrok.connect(8000).public_url
        print(f"\n🚀 ========================================================")
        print(f"🚀 服务已启动！请复制下面的 Public URL 到前端：")
        print(f"👉 {public_url}")
        print(f"📄 API 文档地址: {public_url}/docs")
        print(f"🚀 ========================================================\n")
    except Exception as e:
        print(f"Ngrok 连接失败: {e}")

    # 4. 启动 Uvicorn 后端
    # 我们使用 subprocess 在当前进程中运行，以便我们可以看到日志
    cmd = ["python", "-m", "backend.server"]
    
    try:
        # 使用 subprocess.Popen 实时输出日志
        process = subprocess.Popen(
            cmd,
            stdout=subprocess.PIPE,
            stderr=subprocess.STDOUT,
            text=True,
            bufsize=1
        )
        
        # 实时打印日志
        for line in process.stdout:
            print(line, end="")
            
    except KeyboardInterrupt:
        print("🛑 服务已停止")
        ngrok.kill()